In [1]:
# Проверял, есть ли еще какое-то влияние случайности

# при перезапуске одной модели могут быть незначительные отличия в значениях тотал ревардв в запусках с авариями.

# сид работает нормально. Если заного инициализировать среду и задавать сид, или просто задавать сид и делать резет - результат одинаковый

# для модели обученной на коммите среды 3862ab3b6f6b169712ab07711c1368068e2509f5 сравнивал результаты запусков \
# на этом коммите и на ea3e81a7fb23ee9b4d8946170158772fd4755ce0. Результат практически одинаков. Иногда встречаются отличия +-1, совсем редко +-1.
# только для сравнения этих коммитов надо не забыть поменять множительв if self.finish_position_framestimer > self.frames_per_step * 20:

In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/continuous-grid-arctic/")
sys.path.append("/s/ls4/users/grartem/RL_robots/RL_robotSim/")
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [2]:
os.chdir("/s/ls4/users/grartem/RL_robots/RL_robotSim/")

In [3]:
import time
import gym
import pyhocon
import numpy as np
import pandas as pd
import ray
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ddpg.td3 import TD3Trainer

import MyMisc

/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Couldn't import `requests` library. Be sure to install it on the client side.
Traceback (most recent call last):
  File "/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/ray/tune/web_server.py", line 16, in <module>
    import requests  # `requests` is not part of stdlib.
  File "/s/ls4/users/grartem/.local/lib/python3.8/site-packages/requests/__init__.py", line 43, in <module>
    import urllib3
ModuleNotFoundError: No module named 'urllib3'
Couldn't import `requests` library. Be sure to install it on the client side.


pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
configs = pyhocon.ConfigFactory.parse_file("/s/ls4/users/grartem/RL_robots/RL_robotSim/Configs/FollowerContinuous/PPO_obst.conf")
#CONFIG = configs["ppo_archv3"].as_plain_ordered_dict()
CONFIG = configs["ppo_env4_feats10v7"].as_plain_ordered_dict()
CONFIG["config"]["num_workers"]=1

In [6]:
CONFIG["config"]["env_config"]

OrderedDict([('name', 'Test-Cont-Env-Auto-v0'),
             ('base_env_config',
              OrderedDict([('add_obstacles', True),
                           ('warm_start', 0),
                           ('max_steps', 10000),
                           ('framerate', 5000),
                           ('obstacle_number', 35),
                           ('constant_follower_speed', False),
                           ('early_stopping',
                            OrderedDict([('max_distance_coef', 1.4),
                                         ('low_reward', -300)])),
                           ('random_frames_per_step', [2, 20]),
                           ('leader_speed_regime',
                            OrderedDict([('0', [0.2, 1]),
                                         ('200', 1),
                                         ('1000', [0.5, 1]),
                                         ('1500', 0.75),
                                         ('2000', 0),
                              

# Chose checkpoint

In [9]:
EXPERIMENT_DIR = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4_feats10v7/PPO_continuous-grid_d1217_00000_0_2022-04-11_13-26-15/"
df= pd.read_csv(EXPERIMENT_DIR + "/progress.csv")
#df = df.loc[list(range(0,400,10))]
df = df.loc[list(range(0,100,10))]


df.sort_values(by=["episode_reward_max", "episode_reward_mean", "episode_reward_min"], ascending= False)
print("top max rewards")
print(df.sort_values(by=["episode_reward_max", "episode_reward_mean", "episode_reward_min"], ascending= False).index[:10])
df.sort_values(by=["episode_reward_mean", "episode_reward_max", "episode_reward_min"], ascending= False)
print("top mean rewards")
print(df.sort_values(by=["episode_reward_mean", "episode_reward_max", "episode_reward_min"], ascending= False).index[:10])
df.sort_values(by=[ "episode_reward_min","episode_reward_mean", "episode_reward_max"], ascending= False)
print("top min rewards")
print(df.sort_values(by=[ "episode_reward_min","episode_reward_mean", "episode_reward_max"], ascending= False).index[:10])

df["reward_sum"] = df.loc[:, ["episode_reward_min","episode_reward_mean", "episode_reward_max"]].sum(axis=1)
df.sort_values(by="reward_sum", ascending= False)

top max rewards
Int64Index([40, 50, 80, 70, 90, 60, 30, 20, 10, 0], dtype='int64')
top mean rewards
Int64Index([80, 90, 70, 60, 50, 40, 30, 20, 10, 0], dtype='int64')
top min rewards
Int64Index([50, 60, 80, 90, 0, 70, 10, 40, 30, 20], dtype='int64')


,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,info/learner/default_policy/learner_stats/cur_kl_coeff,info/learner/default_policy/learner_stats/cur_lr,info/learner/default_policy/learner_stats/total_loss,info/learner/default_policy/learner_stats/policy_loss,info/learner/default_policy/learner_stats/vf_loss,info/learner/default_policy/learner_stats/vf_explained_var,info/learner/default_policy/learner_stats/kl,info/learner/default_policy/learner_stats/entropy,info/learner/default_policy/learner_stats/entropy_coeff,reward_sum
50,967.0,166.0,397.834000,453.2500,6,4,204000,0,204000,False,...,0.225,0.00005,521.793679,-0.016280,521.806979,-0.224250,0.013249,2.043042,0.0,1530.834000
80,964.0,-45.5,503.489000,565.2200,7,4,324000,0,324000,False,...,0.225,0.00005,818.423616,-0.018606,818.438735,-0.211026,0.015488,2.263907,0.0,1421.989000
90,953.0,-45.5,495.793000,553.1700,9,4,364000,0,364000,False,...,0.225,0.00005,626.394909,-0.010568,626.403360,-0.347427,0.009410,1.980595,0.0,1403.293000
70,964.0,-50.0,474.251000,513.1500,9,4,284000,0,284000,False,...,0.225,0.00005,859.928167,-0.008481,859.934046,-0.229556,0.011564,1.896323,0.0,1388.251000
60,935.0,-40.0,427.510000,479.8900,6,4,244000,0,244000,False,...,0.225,0.00005,425.103878,-0.013982,425.115151,-0.288715,0.012046,1.935791,0.0,1322.510000
40,972.0,-66.0,375.070000,425.6800,8,4,164000,0,164000,False,...,0.225,0.00005,599.904583,-0.015266,599.917719,-0.037852,0.009446,2.004223,0.0,1281.070000
30,763.5,-70.0,269.563000,322.8100,13,4,124000,0,124000,False,...,0.150,0.00005,221.143309,-0.012651,221.154283,0.443083,0.011185,2.218809,0.0,963.063000
20,681.5,-82.0,160.463000,274.5900,11,4,84000,0,84000,False,...,0.100,0.00005,778.437831,-0.013893,778.450449,0.176234,0.012769,2.306331,0.0,759.963000
10,270.0,-50.7,67.740000,142.4900,23,4,44000,0,44000,False,...,0.100,0.00005,711.756123,-0.009667,711.764934,0.233067,0.008536,2.563152,0.0,287.040000
0,98.9,-48.5,28.190625,116.8125,32,4,4000,0,4000,False,...,0.200,0.00005,832.831590,-0.001210,832.832194,0.004005,0.003037,2.857519,0.0,78.590625


In [10]:
checkpoint_path = EXPERIMENT_DIR + "/checkpoint_000080/checkpoint-80"
table_save_path = EXPERIMENT_DIR + "/checkpoint-80_test.csv"

In [11]:
trainer = ray.rllib.agents.registry.get_trainer_class(CONFIG["run"])(CONFIG["config"])
trainer.restore(checkpoint_path)

env = MyMisc.continuous_env_maker(config=CONFIG["config"]["env_config"])

(pid=9114) 2022-04-11 17:03:52,458	ERROR web_server.py:19 -- Couldn't import `requests` library. Be sure to install it on the client side.
(pid=9114) Traceback (most recent call last):
(pid=9114)   File "/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/ray/tune/web_server.py", line 16, in <module>
(pid=9114)     import requests  # `requests` is not part of stdlib.
(pid=9114)   File "/s/ls4/users/grartem/.local/lib/python3.8/site-packages/requests/__init__.py", line 43, in <module>
(pid=9114)     import urllib3
(pid=9114) ModuleNotFoundError: No module named 'urllib3'
(pid=9114) 2022-04-11 17:03:52,916	WARNING policy_client.py:28 -- Couldn't import `requests` library. Be sure to install it on the client side.


(RolloutWorker pid=9114) pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
(RolloutWorker pid=9114) Hello from the pygame community. https://www.pygame.org/contribute.html


(RolloutWorker pid=9114) /s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/follow_the_leader_continuous_env.py:257: UserWarning: Одновременно заданы и random_frames_per_step и frames_per_step, будет использоваться random_frames_per_step
(RolloutWorker pid=9114)   warn("Одновременно заданы и random_frames_per_step и frames_per_step, будет использоваться random_frames_per_step")
(RolloutWorker pid=9114) /s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=9114)   logger.warn(


(RolloutWorker pid=9114) ===Запуск симуляции номер 0===


(RolloutWorker pid=9114) Failed to create secure directory (/run/user/1556/pulse): No such file or directory
(RolloutWorker pid=9114) ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
(RolloutWorker pid=9114) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
(RolloutWorker pid=9114) ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
(RolloutWorker pid=9114) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
(RolloutWorker pid=9114) ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
(RolloutWorker pid=9114) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
(RolloutWorker pid=9114) ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
(RolloutWorker pid=9114) ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
2022-04-11 17:04:0

# collect test table

In [13]:
history = []
for seed_i in range(100):
    print(seed_i)
    env.seed(seed_i)
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    start_time = time.time()
    while not done:
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs, explore=False)
        # Apply the computed action in the environment.
        obs, reward, done, info = env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
    elapsed_time = time.time() - start_time
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(elapsed_time)
    history.append({
        "seed": seed_i,
        "reward": total_reward,
        "done": done,
        "time": elapsed_time
    })
    history[-1].update(info)
    pd.DataFrame(history).to_csv(table_save_path, index=False, sep=";", encoding="cp1251")
    # Report results.

0
===Запуск симуляции номер 0===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
/s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/utils/misc.py:63: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(v.dot(w) / (np.linalg.norm(v, axis=1) * np.linalg.norm(w)))


Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
00:00:13
1
===Запуск симуляции номер 1===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
/s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/utils/misc.py:63: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(v.dot(w) / (np.linalg.norm(v, axis=1) * np.linalg.norm(w)))
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
00:01:08
2
===Запуск симуляции номер 2===


...

00:01:11
8
===Запуск симуляции номер 8===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:23
9
===Запуск симуляции номер 9===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
/s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/utils/misc.py:63: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(v.dot(w) / (np.linalg.norm(v, axis=1) * np.linalg.norm(w)))
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


KeyboardInterrupt: 

# analyze test table

In [22]:
import pandas as pd
df_1 = pd.read_csv("/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4_featsv2/PPO_continuous-grid_40cc8_00001_1_2022-04-07_00-12-40/checkpoint-330_test.csv", sep=";", encoding="cp1251")
print("total of success or finished by time", len(df_1[df_1["mission_status"].isin(["success", "finished_by_time"])]))
print("total of too far or low reward stops", len(df_1[df_1["agent_status"].isin(["too_far_from_leader", "low_reward"])]))
print("total of leader finished", len(df_1[df_1["leader_status"].isin(["finished"])]))
print("sum of rewards", df_1["reward"].sum())

total of success or finished by time 84
total of too far or low reward stops 11
total of leader finished 83
sum of rewards 54796.10000000001


In [ ]:
df_1 = pd.read_csv("/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4/PPO_continuous-grid_40cc8_00000_0_2022-04-07_00-12-23/checkpoint-240_test.csv", sep=";", encoding="cp1251")

In [13]:
df_1[df_1["mission_status"]=="fail"]

,seed,reward,done,time,mission_status,agent_status,leader_status
0,0,153.0,True,00:00:12,fail,crash,moving
2,2,168.0,True,00:00:14,fail,crash,moving
4,4,204.2,True,00:00:22,fail,too_far_from_leader,moving
5,5,227.0,True,00:00:18,fail,crash,moving
9,9,407.4,True,00:00:49,fail,too_far_from_leader,moving
10,10,105.6,True,00:00:16,fail,too_far_from_leader,moving
12,12,108.8,True,00:00:14,fail,too_far_from_leader,moving
15,15,-55.0,True,00:00:16,fail,low_reward,moving
17,17,104.2,True,00:00:13,fail,too_far_from_leader,moving
19,19,174.0,True,00:00:18,fail,crash,moving


In [33]:
df_1[(df_1["agent_status"].isin(["too_far_from_leader", "low_reward"])) & (df_2["agent_status"].isin(["too_far_from_leader", "low_reward"]))]

,seed,reward,done,time,mission_status,agent_status,leader_status
9,9,-20.0,True,00:00:13,fail,low_reward,moving
35,35,158.5,True,00:00:16,fail,too_far_from_leader,moving
36,36,-35.0,True,00:00:00,fail,low_reward,moving
38,38,-17.0,True,00:00:08,fail,low_reward,moving
47,47,167.0,True,00:00:16,fail,too_far_from_leader,moving
91,91,191.5,True,00:00:17,fail,too_far_from_leader,moving


In [188]:
df_2["leader_status"].unique()

array(['finished', 'moving'], dtype=object)

# record some videos

In [26]:
monitor_env = gym.wrappers.Monitor(env=env,
                                directory= os.path.dirname(checkpoint_path)+"/videos2",
                               video_callable=lambda _:True,
                               force=False,
                              uid="seeds_test", mode="evaluation")
#for seed_i in [9, 35, 36, 38, 44, 47, 59, 89, 91, 97]:
for seed_i in [0, 9, 15, 22, 28, 54]:
    monitor_env.seed(seed_i)
    obs = monitor_env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    start_time = time.time()
    while not done:
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs, explore=False)
        # Apply the computed action in the environment.
        obs, reward, done, info = monitor_env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
obs = monitor_env.reset()

===Запуск симуляции номер 100===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
/s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/utils/misc.py:63: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(v.dot(w) / (np.linalg.norm(v, axis=1) * np.linalg.norm(w)))


Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
Лидер столкнулся с препятствием!
===Запуск симуляции номер 101===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
===Запуск симуляции номер 102===


...

Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
===Запуск симуляции номер 105===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
===Запуск симуляции номер 106===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


In [20]:
monitor_env = gym.wrappers.Monitor(env=env,
                                directory= os.path.dirname(checkpoint_path)+"/videos",
                               video_callable=lambda _:True,
                               force=True,
                              uid="seeds_test", mode="evaluation")
#for seed_i in [9, 35, 36, 38, 44, 47, 59, 89, 91, 97]:
for seed_i in [34]:
    monitor_env.seed(seed_i)
    obs = monitor_env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    start_time = time.time()
    while not done:
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs, explore=False)
        # Apply the computed action in the environment.
        obs, reward, done, info = monitor_env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
    print(total_reward)
obs = monitor_env.reset()

===Запуск симуляции номер 0===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
Лидер столкнулся с препятствием!
АВАРИЯ! -10.0
151.0
===Запуск симуляции номер 1===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
